In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


In [ ]:
## Pruebo agregar otros feats

In [ ]:
ct = pd.crosstab(events_df.person, events_df.staticpage, normalize=True)

In [ ]:
features_df = features_df.join(ct, on='person', how='left')

In [ ]:
##tiro las cols que dan poca importance en conjunto con ev total y quincenas
features_df = features_df.drop(columns=['Conditions',
       'PrivacyEcommerce', 'Quiosks', 'TermsAndConditionsEcommerce',
       'TermsAndConditionsReturnEcommerce', 'black_friday', 'club-trocafone',
       'galaxy-s8', 'how-to-buy', 'how-to-sell', 'trust-trocafone'])

In [ ]:
features_df.columns

In [ ]:
ct = pd.crosstab(events_df.person, events_df.campaign_source, normalize=True)
ct.head()

In [ ]:
features_df = features_df.join(ct, on='person', how='left')

## Cantidad eventos totales por usuario

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad eventos de cada uno

In [74]:
df = events_df.groupby('person').event.value_counts()

In [75]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [76]:
df = df.drop(columns=['lead'])

In [77]:
df['ach*bl'] = (df['ad campaign hit'])*(df['brand listing'])
df['ach*gl'] = (df['ad campaign hit'])*(df['generic listing'])
df['ach*seh'] = (df['ad campaign hit'])*(df['search engine hit'])
df['ach*sp'] = (df['ad campaign hit'])*(df['searched products'])
df['ach*vp'] = (df['ad campaign hit'])*(df['viewed product'])
df['ach*vs'] = (df['ad campaign hit'])*(df['visited site'])

In [78]:
df.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'ach*bl', 'ach*gl',
       'ach*seh', 'ach*sp', 'ach*vp', 'ach*vs'],
      dtype='object', name='event')

In [79]:
df = df.drop(columns=['checkout', 'conversion', 'visited site', 'staticpage', 'generic listing']).head()

In [80]:
features_df = features_df.join(df, on='person')

In [81]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Quincenas

In [10]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [56]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [57]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [58]:
features_df = features_df.join(df, on='person')

## dia con mas entradas

In [11]:
# Los mas frecuentes son martes, miercoles y jueves
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [ ]:
df = events_df.groupby('person').weekday.value_counts()

In [ ]:
df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday']).head()

In [ ]:
df = df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday'])
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

## Support para dia

In [12]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct = ct.astype('bool')

In [13]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.4, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.414098,(Thursday),1
1,0.404466,(Tuesday),1
2,0.416982,(Wednesday),1


In [14]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [15]:
result = columna_por_persona(ct)

In [16]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support   

In [17]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value 

In [18]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38829

In [19]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas

In [20]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38829

In [21]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [22]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,weekday_support
0,4886f805,9,0.0
1,ad93850f,65,0.0
2,0297fc1e,567,0.0


## Support para browser version

In [24]:
ct = pd.crosstab(events_df.person, events_df.browser_version)
ct = ct.astype('bool')

In [31]:
frequent_itemsets = apriori(ct, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.081612,(Chrome 65.0),1
1,0.378118,(Chrome 66.0),1
2,0.050494,(Chrome Mobile 39),1
3,0.077010,(Chrome Mobile 65.0),1
4,0.281418,(Chrome Mobile 66.0),1


In [32]:
result = columna_por_persona(ct)

In [33]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38242

In [34]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38242

In [37]:
d = {'person': lista_personas, 'browser_support': lista_support}
df = pd.DataFrame(data=d)

In [38]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,weekday_support,browser_support
0,4886f805,9,0.0,0.000000
1,ad93850f,65,0.0,0.281418
2,0297fc1e,567,0.0,0.000000


In [ ]:
#Tiro las columnas con menos de 0.04 de importance
features_df = features_df.drop(columns=['q3', 'q1', 'q7', 'q6', 'q5', 'q4', 'q2'])

In [39]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [82]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 26)

In [83]:
train_df.fillna(0, inplace=True)

In [84]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 25)

In [85]:
to_predict.fillna(0, inplace=True)

/home/miki_mustard/envdatos/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [86]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [87]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [88]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## SVM

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
regr = LinearSVR(random_state=0, tol=1e-5)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
preds = regr.predict(X_test)

In [ ]:
preds

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

In [ ]:
from sklearn import linear_model
n_samples, n_features = 10, 5

clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, preds)

In [ ]:
roc_auc_score(y_test, preds)

## Random Forest

In [93]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [89]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [90]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [91]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.5958320773003232

In [ ]:
roc_auc_score(y_test,predictions)

In [94]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_eventos_totales Importance: 0.57
Variable: browser_support      Importance: 0.08
Variable: weekday_support      Importance: 0.05
Variable: q8                   Importance: 0.05
Variable: q9                   Importance: 0.05
Variable: q5                   Importance: 0.03
Variable: q6                   Importance: 0.03
Variable: q7                   Importance: 0.03
Variable: q1                   Importance: 0.02
Variable: q2                   Importance: 0.02
Variable: q3                   Importance: 0.02
Variable: q4                   Importance: 0.02
Variable: q10                  Importance: 0.02
Variable: cant_ach             Importance: 0.0
Variable: cant_bl              Importance: 0.0
Variable: cant_seh             Importance: 0.0
Variable: cant_s_p             Importance: 0.0
Variable: cant_vp              Importance: 0.0
Variable: ach*bl               Importance: 0.0
Variable: ach*gl               Importance: 0.0
Variable: ach*seh              Importance: 0.0


In [ ]:
features_df = features_df.drop(columns=['ach*vs', 'ach*vp', 'ach*sp', 'ach*seh', 'ach*gl', 'ach*bl', 'viewed product', 'searched products', 'search engine hit', 'brand listing', 'ad campaign hit'])

In [ ]:
rf = RandomForestRegressor()
from sklearn.model_selection import GridSearchCV
#Grid search
rf_param_grid = {
    'max_depth':[None],
    'max_features':[1,3,10],
    #'min_sample_split':[2,3],
    'min_samples_leaf':[1,3,10],
    'n_estimators':[50,75,100],
}

rf_gs = GridSearchCV(rf, param_grid=rf_param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=2, verbose=1, return_train_score=True)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
rf_gs.best_estimator_

In [ ]:
rf_gs.best_score_

In [ ]:
preds_rf_gs = rf_gs.predict(to_predict.drop(columns=['person']))

## XGBoost

In [52]:
import xgboost as xgb
from xgboost import XGBRegressor
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [95]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [96]:
preds = xg_reg.predict(X_test)

In [97]:
roc_auc_score(y_test,preds)

0.6964549076045911

In [ ]:
roc_auc_score(y_test,preds)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid =  {
        'min_child_weight': [1, 5, 10],
        'gamma': [1, 1.5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3],
        'colsample_bylevel': [0.8, 0.9],
        'max_depth': [9, 11, 13],
        'base_score': [0.1, 0.2],
        'n_estimators': [50, 70, 80, 90]
        }

In [ ]:
clf = GridSearchCV(xg_reg, param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_score_

In [ ]:
preds_gs = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, preds_gs)

In [ ]:
preds_xgb = xg_reg.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_xgb.shape

In [ ]:
preds_xgb

In [ ]:
preds_rf_gs.shape

In [ ]:
combinado_xgb_y_rf = preds_rf_gs*0.2 + preds_xgb*0.8
combinado_xgb_y_rf

In [ ]:
roc_auc_score(y_test, combinado_xgb_y_rf)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params ={
    'C':[0.8, 0.9, 1.0],
    'max_iter':[50, 70, 100],
    'n_jobs':[1, 2],
    'random_state':[1,2,3]
}

In [ ]:
l_reg = LogisticRegression(random_state=0, solver='sag')

In [ ]:
clf = GridSearchCV(l_reg, params, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict_proba(X_test)[:,1] 

In [ ]:
clf.score(X_test, y_test)

In [ ]:
roc_auc_score(y_test, preds)

## CSV a publicar

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person
to_publish.shape

In [ ]:
to_publish['label'] = combinado_xgb_y_rf
to_publish.shape

In [ ]:
to_publish.head()

In [ ]:
to_publish.loc[ to_publish.label > 1 ]

In [ ]:
to_publish.loc[ to_publish.label < 0 ]

In [ ]:
to_publish.to_csv('29_11_2.csv', index=False)